<p style="font-family: Arial; font-size:2.5em;color:crimson;">
    <b>1) Libraries and Dataset</b> 
 </p>

## Import Libraries

In [ ]:
%matplotlib inline
## General Libraries and Visualization Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pgeocode 
import matplotlib.gridspec as gridspec
import warnings
warnings.filterwarnings('ignore')
## Machine Learning Libraries
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn import preprocessing
from sklearn.metrics import classification_report,confusion_matrix,precision_score,jaccard_score,recall_score,f1_score

## Import and Read Dataset

In [ ]:
df = pd.DataFrame(pd.read_csv('C:/Users/Gagan Sharma/OneDrive/Desktop/Data/Bank_Loan_Granting.csv'))
df     

### Samples
    - This dataset has 5000 sample in 14 columns.
### Features :
     ID : customer ID
     Age : age of customer
     Experience : the number of years that customer has experience
     Income : the customer's annual earnings
     ZIP Code : the postal code of customer's location
     Family : the number of people in customer's family
     CCAvg : the average of monthly credit card spending
     Education : the level of customer's Education(undergraduate,graguate,professional)
     Mortgage : the Value of customer's mortgage(thing that the customer uses it as a guarantee to the bank)
     Securities Account : the customer has a security account or not(it's also known as an investment account)
     CD Account : the customer has a CD account or not(a type of savings account that lets the customer earn interest on a 
     fixed amount of money for a fixed period of time)
     Online : the customer uses the bank's online services or not
     CreditCard : the customer uses the bank's credit card or not
### Target
    - Personal Loan : the customer is granted loans or not

<p style="font-family: Arial; font-size:2.5em;color:crimson;">
    <b>2) Data cleaning</b> 
 </p>

## data information

In [ ]:
df.info()

## Checking missing values

In [ ]:
df.isna().sum()

In [ ]:
df['CCAvg'] = df['CCAvg'].str.replace("/" , '.').astype(np.float64)
df

In [ ]:
df.info()

## Checking duplicated data

In [ ]:
print('Number of duplicated data : ' , len(df[df.duplicated()]))

In [ ]:
df.describe()

## Replace negetive values of Exprience with Absolute of them

In [ ]:
df['Experience'] = df['Experience'].abs()
df.describe()

In [ ]:
df['Income'] = df['Income']/12
df

## Remove a useless column

In [ ]:
df = df.drop(['ID'],axis=1)
df

## The Target Overview

In [ ]:
print('*'*120)
print('number of customers : ',df.shape[0])
print('The number of Personal Loan = 0 : ',df['Personal Loan'].value_counts()[0])
print('The number of Personal Loan = 1 : ',df['Personal Loan'].value_counts()[1])
print('*'*120)

plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
sns.countplot(data=df, x='Personal Loan',palette = "Dark2")
plt.title("count plot for Target(Personal Loan)")
plt.subplot(1,2,2)
plt.pie([df['Personal Loan'].value_counts()[0],df['Personal Loan'].value_counts()[1]],
        labels=['Personal Loan = 0','Personal Loan = 1'],autopct='%1.1f%%',colors=['teal','chocolate'])
plt.title("Percent of Personal Loan")
plt.tight_layout()
plt.show()

In [ ]:
descrete_col = df[['Family','Education','Securities Account','CD Account','Online','CreditCard']]
continuous_col = df[['Age','Experience','Income','CCAvg','Mortgage']]

In [ ]:
for col in descrete_col :
    print('%s : ' %col , df[col].unique())
    
ax = sns.stripplot(data=descrete_col)
ax.set_xticklabels(ax.get_xticklabels(),rotation=30)
plt.ylabel('Class values of descrete Features')
plt.xlabel('Name of descrete Features')
plt.title('OverView of classes of descrete Features')
plt.show()

In [ ]:
print('\n')
print('*'*120)

plt.figure(figsize=(12,1))
ax = sns.boxplot(x=df['ZIP Code'],color="lime", linewidth=.75)
plt.annotate('Noise',xy=(10500,0),xytext=(15000,0),arrowprops=dict(facecolor='red'))
ax.set_title('Distribution of ZIP Code')
plt.show()

fig = px.strip(df[df["ZIP Code"]>80000],
         x="Personal Loan",
         y="ZIP Code",
         stripmode='overlay')
fig.update_layout(annotations=[
        dict(
            x=0,
            y=96900,
            text="ZIP Codes 96651 Must Be Checked\n for analysis",
            textangle=0,
            ax=150,
            ay=-50,
            font=dict(
                color="red",
                size=14
            ),
            arrowcolor="red",
            arrowsize=3,
            arrowwidth=1,
            arrowhead=1)])
       
fig.show()

print('The number of categories in ZIP Code is : ',len(df['ZIP Code'].unique()))
print('There is a noise in zipcode feature, it is the smallest value of it :')
display(df['ZIP Code'].nsmallest(3).to_frame())
print('The range of ZIP Code without this noise is : (%d , %d)'%(df[df['ZIP Code']>80000]['ZIP Code'].min(),
                                           df[df['ZIP Code']>80000]['ZIP Code'].max()))

In [ ]:
display(continuous_col.describe().T)
ax = sns.stripplot(data=continuous_col)
ax.set_xticklabels(ax.get_xticklabels(),rotation=30)
plt.ylabel('Range of Continuous Features')
plt.xlabel('Name of Continuous Features')
plt.show()

In [ ]:
for col in continuous_col.columns:
    plt.figure(figsize=(12,0.5))
    ax = sns.boxplot(x=df[col],color="greenyellow", linewidth=.75)
    ax.set_title(f'Distribution of {col} feature')
    plt.show()

In [ ]:
## df1 is just for visualizations related to zipcodes
df1 = df.copy()
df = df.drop(['ZIP Code'],axis=1)
df1.drop(index=384,inplace=True)
df1.reset_index(drop=True, inplace =True)
df1

## Insert Latitude, Longitude, County and Place name to df1 according to zipcode

In [ ]:
nomi = pgeocode.Nominatim('us')
df2 = []
df3 = []
df4 = []
df5 = []

for zipCode in df1['ZIP Code'] :
    a = nomi.query_postal_code(zipCode)
    df2.append(a['latitude'])
    df3.append(a['longitude'])
    df4.append(a['county_name'])
    df5.append(a['place_name'])
    
df1['Lat'] = pd.DataFrame(df2)
df1['Lon'] = pd.DataFrame(df3)
df1['County'] = pd.DataFrame(df4)
df1['Place'] = pd.DataFrame(df5)
df1

In [ ]:
df1.describe(include='all')

In [ ]:
df1[df1['Lat'].isna()]['ZIP Code'].unique()

In [ ]:
miss_zipcodes = [92717,92634,92709]
miss_zipcodes_info = [[33.6462,-117.8397,'Orange','Irvine'],[33.8739,-117.9028,'Orange','Fullerton']
                      ,[33.67,-117.73,'Orange','Irvine']]

for i,zcode in enumerate(miss_zipcodes) :
    for j,info in enumerate(['Lat','Lon','County','Place']) :
        df1.loc[df1['ZIP Code']==zcode,info]=miss_zipcodes_info[i][j]

In [ ]:
df1.isna().sum()

In [ ]:
df1 = df1.dropna()
display(df1.isna().sum())
display(df1)

In [ ]:
fig = px.strip(df1,
         x="Personal Loan",
         y="ZIP Code",
         stripmode='overlay')
fig.update_layout(annotations=[
        dict(
            x=0,
            y=96900,
            text="ZIP Codes 96651 have been removed as unknown zipcodes",
            textangle=0,
            ax=150,
            ay=-50,
            font=dict(
                color="red",
                size=14
            ),
            arrowcolor="red",
            arrowsize=3,
            arrowwidth=1,
            arrowhead=1)])
       
fig.show()

<p style="font-family: Arial; font-size:2.5em;color:crimson;">
    <b>3) Data analysis and visualization</b> 
 </p>

In [ ]:
plt.figure(figsize=(15,10))
for i,col in enumerate(list(descrete_col.columns)):
    plt.subplot(2,3,i+1)
    sns.countplot(data=df, x=col)
    plt.title(f"count plot for $\\mathbf{{{col}}}$")
plt.tight_layout()
plt.show()
#############################################################
descrete_cols_categories=[]
descrete_cols_categories_count=[]

for col in list(descrete_col.columns):
    pie_name1=[]
    pie_value1=[]
    for j in range(len(descrete_col[col].unique())) :
        pie_name1.append(descrete_col[col].unique()[j])
        pie_value1.append(descrete_col[descrete_col[col]==pie_name1[j]][col].count())
    descrete_cols_categories.append(pie_name1)
    descrete_cols_categories_count.append(pie_value1)

print('*'*120)
print('Descrete columns name : \n' , list(descrete_col.columns))
print('Descrete columns categories : \n' , descrete_cols_categories)   
print('Descrete columns categories count : \n' , descrete_cols_categories_count)
print('\n')
print('*'*120)
#############################################################
rows = 3
cols = 2

# Create subplots
fig = make_subplots(rows=rows, cols=cols, specs=[[{"type": "pie"}, {"type": "pie"}],[{"type": "pie"}, {"type": "pie"}],
                                           [{"type": "pie"}, {"type": "pie"}]])


for i in range(rows * cols) :
    fig.add_trace(go.Pie(labels=descrete_cols_categories[i], values=descrete_cols_categories_count[i], name=descrete_col.columns[i]),
                  row=int(np.ceil((i+1)/2)), col=(i%2)+1)


fig.update_layout(margin=dict(t=35, b=35, l=35, r=35))
fig.update(layout_title_text='Percent Of values in descrete columns',
           layout_showlegend=True)

fig = go.Figure(fig)
fig.show()

In [ ]:
plt.figure(figsize=(15,10))
for i,col in enumerate(list(descrete_col.columns)):
    plt.subplot(2,3,i+1)
    sns.countplot(data=df, x=col,hue='Personal Loan')
    plt.title(f"count plots for $\\mathbf{{{col}}}$")
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(18,12))
for i,col in enumerate(continuous_col.columns):
    plt.subplot(3,2,i+1)
    sns.histplot(x=df[col],hue = df['Personal Loan'],kde=True)
    plt.title(f"Personal loan Vs $\\mathbf{{{col}}}$",fontsize=18)

plt.tight_layout()
plt.show()

#############################################################################
print('\n')
print('*'*120)

plt.figure(figsize=(15,10))
for i,col in enumerate(continuous_col.columns):
    plt.subplot(3,2,i+1)
    color=['blue','orange']
    for j in list(df['Personal Loan'].unique()) :
        sns.kdeplot(df[df['Personal Loan']==j][col],color=color[j],fill=True,label=j)
        plt.legend()
    plt.title(f"Distribution of Personal Loan on $\\mathbf{{{col}}}$ feature")
plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize = (15,10),constrained_layout = True, dpi=100)
GridSpec = gridspec.GridSpec(ncols=1, nrows=2, figure= fig)
 
subfigure = fig.add_subfigure(GridSpec[0,:])
# Adding subplots to subfigure 
subplots = subfigure.subplots(1,2)

total_names=[["Customers' \nincome <= 8", "Customers' \nincome > 8"],["Mortgage = 0", "Mortgage > 0"]]
total_size=[[3788,1212],[3462,1538]]
sub_names=["Personal\n Loan = 1","Personal\n Loan = 0","Personal\n Loan = 1","Personal\n Loan = 0"]
sub_size=[[42,3746,438,774],[312,3150,168,1370]]

# Create colors
c1, c2, c3, c4=[plt.cm.Purples, plt.cm.Greens, plt.cm.Blues, plt.cm.Oranges]
colors = [[c1(0.8), c2(0.8)] ,[c1(0.6), c1(0.4), c2(0.6), c2(0.4)],[c3(0.8), c4(0.8)],[c3(0.6), c3(0.4), c4(0.6), c4(0.4)]]

for i,col in enumerate(['Income','Mortgage']):
    subplots[i].axis('equal')
    pie1, _ = subplots[i].pie(total_size[i], radius=0.8, labels=total_names[i],textprops={'fontsize': 15},
                              colors= colors[i*2])
    plt.setp( pie1, width=0.3, edgecolor='white')
    # Second Ring (Inside)
    pie2, _ = subplots[i].pie(sub_size[i], radius=0.85-0.3, 
    labels=sub_names, labeldistance=0.7,textprops={'fontsize': 14}, colors=colors[i*2+1])
    plt.setp( pie2, width=0.25, edgecolor='white')
    plt.margins(0,0)
    subplots[i].set_title(f'$\\mathbf{{{col}}}$ Vs Personal Loan',fontsize=18)
    
plt.show()

print('\n')
print('*'*120)

sns.histplot(x=df[df['Mortgage']>0]['Mortgage'],hue = df['Personal Loan'],kde=True)
plt.title(f'Personal loan Vs $\\mathbf{{{"Mortgage>0"}}}$')

plt.show()

In [ ]:
plt.figure(figsize=(8,5))
sns.pairplot(df[['Age','Experience','Income','CCAvg','Mortgage','Personal Loan']],hue='Personal Loan')
plt.tight_layout()
plt.show()
print('*'*125)

In [ ]:
display(df[['Personal Loan','Income','CCAvg','Mortgage']].corr())
display(df[['Income','CCAvg','Mortgage']].corr())


plt.figure(figsize=(8,6))
for i,col in enumerate(['CCAvg','Mortgage']):
    plt.subplot(1,2,i+1)
    sns.scatterplot(data=df, x=col,y='Income',hue='Personal Loan')
    plt.title(f"$\\mathbf{{{col}}}$ Vs Income")
plt.tight_layout()
plt.show()

In [ ]:
fig = px.scatter_3d(df, x='CCAvg', y='Income', z='CD Account',
              color='Personal Loan')
fig.show()
####################################################################
fig = px.scatter_3d(df, x='Mortgage', y='Income', z='CD Account',
              color='Personal Loan')
fig.show()

In [ ]:
plt.figure(figsize=(10,3))
plt.subplot(1,2,1)
sns.kdeplot(df1['ZIP Code'],fill=True,color='blue',label='all Customers')
sns.kdeplot(df1[df1['Personal Loan']==1]['ZIP Code'],fill=True,color='orange',label='customers with\n personal loan=1')
plt.legend(loc='upper left')
plt.subplot(1,2,2)
ax = sns.boxplot(y=df1['ZIP Code'],x=df1['Personal Loan'],color="green", linewidth=0.75)
ax = sns.stripplot(y=df1['ZIP Code'],x=df1['Personal Loan'],color="orange",linewidth=0.75)
ax.yaxis.tick_right()
ax.yaxis.set_label_position('right')

In [ ]:
fig = px.scatter_mapbox(
    df1 ,
    lat="Lat",
    lon="Lon",
    hover_name="County",
    color = "Personal Loan",
    color_continuous_scale = ["crimson",'darkblue'],
    hover_data=['Place','Income' , 'Mortgage' , "CreditCard"],
    zoom=10,
    height=400
    
)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.update_layout(mapbox_bounds={"west": -200, "east": 80, "south": -20, "north": 90})
fig.show()